In [21]:
from striprtf.striprtf import rtf_to_text
import re
from glob import glob

In [22]:
filenames2 = glob("news*.RTF")
print(filenames2)

['news_corona_2.RTF', 'news_corona_3.RTF', 'news_corona_1.RTF']


In [23]:
rtf_stringX = []
for fn in filenames2:
    with open(fn) as f:
        artikel = f.read()
        rtf_stringX.append(f)

In [14]:
# read the files in
filenames = ["news_corona_" + str(i) + ".RTF" for i in range(1, 4) ]
rtf_string  = [ open(f).read() for f in filenames ]

In [24]:
print(rtf_string[0])

{\rtf1\ansi\ansicpg1252\uc0\stshfdbch0\stshfloch0\stshfhich0\stshfbi0\deff0\adeff0{\fonttbl{\f0\froman\fcharset0\fprq2{\*\panose 02020603050405020304}Times New Roman;}{\f1\froman\fcharset2\fprq2{\*\panose 05050102010706020507}Symbol;}{\f2\fswiss\fcharset0\fprq2{\*\panose 020b0604020202020204}Arial;}}{\colortbl;\red0\green119\blue204;\red255\green255\blue255;
\red0\green0\blue0;\red118\green118\blue118;}{\stylesheet{\s0\snext0\sqformat\spriority0\aspalpha\aspnum\adjustright\ltrpar\li0\lin0\ri0\rin0\ql\faauto\rtlch\afs24\ltrch\fs24 Normal;}{\s1\sbasedon0\snext0\styrsid15694742\sqformat\spriority0\keepn\sb240\sa60\aspalpha\aspnum\adjustright\outlinelevel0
\nowidctlpar\ltrpar\li0\lin0\ri0\rin0\ql\faauto\rtlch\ab\af2\afs32\ltrch\b\fs32\f2\kerning32 heading 1;}{\*\cs10\additive\ssemihidden\spriority0 Default Paragraph Font;}{\*\ts11\tsrowd\snext11\ssemihidden\spriority0\aspalpha\aspnum\adjustright\ltrpar\li0\lin0\ri0\rin0
\ql\faauto\tsvertalt\tsbrdrl\tsbrdrr\tsbrdrt\tsbrdrb\tsbrdrdgr\tsbrdrd

In [27]:
# convert the files from rtf to string format
text = [rtf_to_text(i) for i in rtf_string]

# replace line breaks and split articles

splitted_text = [i.replace("\n", " ").split("End of Document  ") for i in text]

In [26]:
len(text)

3

In [31]:
type(splitted_text[0])

list

In [12]:
print(text[0])





de Volkskrant
3 maart 2020 dinsdag


Copyright 2020 De Persgroep Nederland BV All Rights Reserved


Section: Ten Eerste; Blz. 10
Length: 142 words
Byline: ANP
Highlight: WHO


Body


Het aantal nieuwe gevallen van het gevreesde nieuwe coronavirus uit Wuhan is in de afgelopen 24 uur negen keer hoger geweest buiten China dan in dat land zelf. Dit zei de chef van WHO (Wereldgezondheidsorganisatie) Tedros Adhanom Ghebreyesus maandag in Genève. De uitbraak in Italië, Iran, Japan en Zuid-Korea is nu onze grootste zorg, aldus Tedros. En de epidemie in Zuid-Korea kan nog steeds worden bedwongen door waakzaamheid, aldus Tedros die zei dat een speciale WHO-missie in Iran is aangekomen om bij de bestrijding van het coronavirus te helpen. Duitsland, Frankrijk en Groot-Brittannië hebben 5 miljoen euro hulp aangeboden. Er zijn in Iran al 66 patiënten overleden aan het virus. De gezondheidszorg in Iran worstelt al jaren met vooral Amerikaanse sancties die medische hulp aan patiënten bemoeilijken.

In [18]:
def parse_nexis_uni(news_string):
    ''' parses strings (nexis news articles), so that the title, date and full text are extracted. '''

    parsed_results = []
    has_found_title = False
    for line in news_string:

        # newspaper title
        matchObj1=re.match(" +([a-zA-Z\s]+?) \d+",line)
        if not has_found_title:
            if matchObj1:
                newspaper = matchObj1.group(1)
                has_found_title = True
            else:
                newspaper = "NaN"

        # date
        matchObj2 = re.match(r".*(\d{1,2}) ([jJ]anuari|[fF]ebruari|[mM]aart|[aA]pril|[mM]ei|[jJ]uni|[jJ]uli|[aA]ugustus|[sS]eptember|[Oo]ktober|[nN]ovember|[dD]ecember) (\d{4}).*", line)
        if matchObj2:
            day = matchObj2.group(1)
            month = matchObj2.group(2)
            year = matchObj2.group(3)
            date = (day, month, year )
        else:
            date = "NaN"

        # page
        p = re.findall(r"Blz.*?([0-9]+)", line)
        if p:
            page = p[0]
            
        # full text
        matchObj3=re.match(".*Body(.*) (Bekijk pagina|Classification)",line)
        if matchObj3:
            text = matchObj3.group(1).strip()
        else:
            text = "NaN"

        parsed_results.append( {'newspaper': newspaper,
                       'date' : date,
                       'text': text,
                               "page" : page} )

    return parsed_results

In [19]:
results = []
for document in splitted_text:
    results.extend(parse_nexis_uni(document))

In [34]:
from pprint import pprint
pprint(results)

[{'date': ('3', 'maart', '2020'),
  'newspaper': 'de Volkskrant',
  'page': '10',
  'text': 'Het aantal nieuwe gevallen van het gevreesde nieuwe coronavirus uit '
          'Wuhan is in de afgelopen 24 uur negen keer hoger geweest buiten '
          'China dan in dat land zelf. Dit zei de chef van WHO '
          '(Wereldgezondheidsorganisatie) Tedros Adhanom Ghebreyesus maandag '
          'in Genève. De uitbraak in Italië, Iran, Japan en Zuid-Korea is nu '
          'onze grootste zorg, aldus Tedros. En de epidemie in Zuid-Korea kan '
          'nog steeds worden bedwongen door waakzaamheid, aldus Tedros die zei '
          'dat een speciale WHO-missie in Iran is aangekomen om bij de '
          'bestrijding van het coronavirus te helpen. Duitsland, Frankrijk en '
          'Groot-Brittannië hebben 5 miljoen euro hulp aangeboden. Er zijn in '
          'Iran al 66 patiënten overleden aan het virus. De gezondheidszorg in '
          'Iran worstelt al jaren met vooral Amerikaanse sanct

In [35]:
import json
with open("myparsedfile.json", mode="w") as fo:
    json.dump(results, fo)

In [36]:
allbodies = [e["text"] for e in results]

In [37]:
allbodies

['Het aantal nieuwe gevallen van het gevreesde nieuwe coronavirus uit Wuhan is in de afgelopen 24 uur negen keer hoger geweest buiten China dan in dat land zelf. Dit zei de chef van WHO (Wereldgezondheidsorganisatie) Tedros Adhanom Ghebreyesus maandag in Genève. De uitbraak in Italië, Iran, Japan en Zuid-Korea is nu onze grootste zorg, aldus Tedros. En de epidemie in Zuid-Korea kan nog steeds worden bedwongen door waakzaamheid, aldus Tedros die zei dat een speciale WHO-missie in Iran is aangekomen om bij de bestrijding van het coronavirus te helpen. Duitsland, Frankrijk en Groot-Brittannië hebben 5 miljoen euro hulp aangeboden. Er zijn in Iran al 66 patiënten overleden aan het virus. De gezondheidszorg in Iran worstelt al jaren met vooral Amerikaanse sancties die medische hulp aan patiënten bemoeilijken.  Bekijk de oorspronkelijke pagina:',
 'Bekijk het voor de grap eens vanuit het coronavirus zelf. Eerst China, daarna Zuid-Korea, nu Italie, Frankrijk, Nigeria, Verenigde Staten: binnen